# Joint Ancestral State Reconstruction for the Aculeate Hymenoptera Silk Fibroins
This notebook fits a CNFGTR model that includes a gap state then performs joint ancestral state reconstruction using the algorithm in Pupko et al. (Mol. Biol. Evol. 17(6):890–896. 2000).
### Setup
Note that the cogent that is required here is from [a special PyCogent](https://github.com/BenKaehler/pycogent/tree/codon_gaps).

`gapped` is the Python module in this repo.

In [1]:
import cogent
from numpy import allclose

import gapped

### Load the alignment and the tree

In [2]:
aln = cogent.LoadSeqs('../results/aligned_dna.fasta', moltype=cogent.DNA)
tree = cogent.LoadTree('../data/tree.nwk')

In [3]:
print tree.asciiArt()

                                        /-AmelF2
                              /twoBeesF2
                    /threeBeesF2        \-AdorF2
                   |         |
          /rootF2--|          \-BterF2
         |         |
         |         |          /-OsmaF2
         |          \twoAntsF2
         |                    \-MforF2
         |
         |                              /-AmelF3
         |                    /twoBeesF3
         |          /threeBeesF3        \-AdorF3
         |         |         |
         |-rootF3--|          \-BterF3
         |         |
-root----|         |          /-OsmaF3
         |          \twoAntsF3
         |                    \-MforF3
         |
         |                                        /-AmelF1
         |                              /twoBeesF1
         |                    /threeBeesF1        \-AdorF1
         |                   |         |
         |          /rootF1--|          \-BterF1
         |         |         |
         |

### Fit CNFGTR with gaps

In [4]:
model = gapped.CNFGTR(optimise_motif_probs=True, model_gaps=True)
cnfgtr = model.makeLikelihoodFunction(tree)
cnfgtr.setAlignment(aln)
with cnfgtr.updatesPostponed():
    for param in cnfgtr.getParamNames():
        if param != 'length':
            cnfgtr.setParamRule(param, is_independent=False)
        if '/' in param:
            cnfgtr.setParamRule(param, upper=20., lower=0.05)
cnfgtr.setParamRule('length', is_independent=True, upper=20.)
cnfgtr.optimise(local=True, show_progress=False, limit_action='raise')

/Users/benkaehler/Projects/pycogent-bk/cogent/evolve/parameter_controller.py:241: UserWarning: Used mean of 38 mprobs values
  is_independent)


In [5]:
print cnfgtr

Likelihood Function Table
   A/C       A/G       A/T       C/G       C/T     indel     omega
------------------------------------------------------------------
0.8133    2.3215    1.1722    0.8220    1.9386    0.1436    0.3396
------------------------------------------------------------------
       edge         parent    length
------------------------------------
     AmelF2      twoBeesF2    0.0793
     AdorF2      twoBeesF2    0.0941
  twoBeesF2    threeBeesF2    0.3970
     BterF2    threeBeesF2    0.5936
threeBeesF2         rootF2    1.6002
     OsmaF2      twoAntsF2    0.6853
     MforF2      twoAntsF2    0.7076
  twoAntsF2         rootF2    1.5582
     rootF2           root    0.8846
     AmelF3      twoBeesF3    0.1095
     AdorF3      twoBeesF3    0.0762
  twoBeesF3    threeBeesF3    0.5867
     BterF3    threeBeesF3    0.4855
threeBeesF3         rootF3    0.9854
     OsmaF3      twoAntsF3    0.5207
     MforF3      twoAntsF3    0.7207
  twoAntsF3         rootF3    1.4373
   

### Write out the CNFGTR tree for plotting using another application

In [6]:
for node in cnfgtr.tree.getEdgeVector(include_root=False):
    node.Length = cnfgtr.getParamValue('length', edge=node.Name)
cnfgtr.tree.writeToFile('../results/cnfgtr.nwk',with_distances=True)

### Perform the joint ancestral state reconstruction

In [7]:
anc_aln = gapped.joint(cnfgtr)

### Write out the ancestral state reconstructions as an alignment

In [8]:
anc_aln.writeToFile('../results/joint.fasta')